# Pairs Trading Analysis and Test

In [1]:
%%capture
#!pip install statsmodels;
#!pip install plotly;
#!pip install yfinance;
#!pip install vectorbt;

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import yfinance as yf
import vectorbt as vbt

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

In [4]:
#Download data
coca = yf.download('KO', start='2020-01-01', end='2022-12-31')

#Plot prices
fig = go.Figure()
fig.add_trace(go.Scatter(name='KO', x=coca.index, y=coca['Adj Close']))
fig.update_layout(title_text='KO Prices (USD)', template='simple_white', height=500, width=800)
fig.show()

[*********************100%***********************]  1 of 1 completed


In [5]:
# Calculate price moving average (non-stationary)
coca['mean'] = coca['Adj Close'].rolling('360D').mean()
fig = go.Figure()
fig.add_trace(go.Scatter(name='KO Price', x=coca.index, y=coca['Adj Close']))
fig.add_trace(go.Scatter(name='KO Price 360D Moving Average', x=coca.index, y=coca['mean']))
fig.update_layout(title_text='KO Prices (USD)', template='simple_white', height=500, width=1100)
fig.show()

In [6]:
# Calculate and plot returns
coca['returns'] = coca['Adj Close'].pct_change()*100
coca['returns MA 360D'] = coca['returns'].rolling('360D').mean()
fig = go.Figure()
fig.add_trace(go.Scatter(name='KO Returns', x=coca.index, y=coca['returns']))
fig.add_trace(go.Scatter(name='KO Returns 360D Moving Average', x=coca.index, y=coca['returns MA 360D']))
fig.update_layout(title_text='KO Returns (%)', template='simple_white', height=500, width=1100)
fig.show()

## Test series stationarity (Augmented Dickey-Fuller test)

In [7]:
#Price series stationarity p-value

resultado = adfuller(coca['Adj Close'])
resultado[1]

0.757900305206254

In [8]:
#Return series stationarity p-value

coca.dropna(inplace=True)
resultado2 = adfuller(coca['returns'])
resultado2[1]

1.2056354978794177e-13

In [9]:
print("Prices - Augmented Dickey-Fuller test p-value:", round(resultado[1], 4))
print("Returns - Augmented Dickey-Fuller test p-value:", round(resultado2[1], 15))

Prices - Augmented Dickey-Fuller test p-value: 0.7579
Returns - Augmented Dickey-Fuller test p-value: 1.21e-13


## Cointegration

In [10]:
# Define assets and download data
asset1 = 'KO'
asset2 = 'PEP'

tickers = [asset1, asset2]

start ='2020-01-01'
end='2022-12-31'

assets = pd.DataFrame()

for i in tickers:

  df = yf.download(i, start=start, end=end)['Adj Close']
  df.rename(i, inplace=True)
  assets = pd.concat([assets,df], axis=1)
  assets.index.name='Date'

assets

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,KO,PEP
Date,,
2020-01-02,49.307655,123.187531
2020-01-03,49.038643,123.015182
2020-01-06,49.020721,123.486801
2020-01-07,48.644119,121.545883
2020-01-08,48.733780,122.171654
...,...,...
2022-12-23,62.855492,179.780746
2022-12-27,63.239597,180.579727
2022-12-28,62.609272,179.277679


In [11]:
# Plot prices in line chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=assets.index, y=assets[asset1], name=asset1))
fig.add_trace(go.Scatter(x=assets.index, y=assets[asset2], name=asset2))
fig.update_layout(title_text='Prices (USD)', template='simple_white', width=1100)
fig.show()

In [12]:
# Plot prices in scatter chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=assets[asset1], y=assets[asset2], mode='markers'))
fig.update_layout(title_text='KO Prices x PEP Prices', template='simple_white')
fig.show()

In [13]:
# Plot returns in scatter chart
returns = assets.pct_change()

fig = go.Figure()
fig.add_trace(go.Scatter(x=returns[asset1], y=returns[asset2], mode='markers'))
fig.update_layout(title_text='Retornos', template='simple_white')
fig.show()

### Testing Cointegration

In [14]:
# Run augmented engle granger test and print p-value
assets.dropna(inplace=True)
score, pvalue, _ = coint(assets[asset1], assets[asset2])
print("Augmented Engle-Granger test p-value:", round(pvalue, 4))

Augmented Engle-Granger test p-value: 0.0934


### Calculate the residuals of the regression

In [15]:
# Run linear regression and calculate regression beta

X1 = assets[asset1]
X2 = assets[asset2]

X1 = sm.add_constant(X1)
regression = sm.OLS(X2, X1).fit()
X1 = X1[asset1]
beta = regression.params[asset1]

regression.params

const    6.114310
KO       2.667431
dtype: float64

In [16]:
assets_figure = assets.copy()
assets_figure['regression'] = regression.params[0] + regression.params[1]*assets_figure[asset1]
assets_figure

,KO,PEP,regression
Date,,,
2020-01-02,49.307655,123.187531,137.639056
2020-01-03,49.038643,123.015182,136.921483
2020-01-06,49.020721,123.486801,136.873679
2020-01-07,48.644119,121.545883,135.869119
2020-01-08,48.733780,122.171654,136.108283
...,...,...,...
2022-12-23,62.855492,179.780746,173.776968
2022-12-27,63.239597,180.579727,174.801543
2022-12-28,62.609272,179.277679,173.120194


In [17]:
# Plot prices with regression line
fig = go.Figure()
fig.add_trace(go.Scatter(x=assets_figure[asset1], y=assets_figure[asset2], mode='markers', name='KO x PEP'))
fig.add_trace(go.Scatter(x=assets_figure[asset1], y=assets_figure['regression'], mode='lines', name='Regression'))
fig.update_layout(title_text='KO Prices x PEP Prices (USD)', template='simple_white', width=1100)
fig.show()

In [18]:
# Calculate regression residual
spread = X2 - beta * X1

In [19]:
# Plot residuals in line chart as a time series
fig = go.Figure()
fig.add_trace(go.Scatter(x=spread.index, y=spread, name='Spread'))
fig.update_layout(title_text='Residual Time Series (USD)', template='simple_white', width=1100)
fig.show()

In [20]:
# Run augmented dickey fuller test and print p-value
test = adfuller(spread)
print("Residuals - Augmented Dickey-Fuller Test p-value:", round(test[1], 4))

Residuals - Augmented Dickey-Fuller Test p-value: 0.0433


### Calculate Z-score

<img src="https://latex.codecogs.com/svg.image?Z&space;=&space;\frac{x&space;-&space;mu&space;}{\sigma&space;}" title="https://latex.codecogs.com/svg.image?Z = \frac{x - mu }{\sigma }" />

In [21]:
# Calculate z-score of residuals and plot in line chart
z_score = (spread- spread.mean())/np.std(spread)

fig = go.Figure()
fig.add_trace(go.Scatter(x=z_score.index, y=z_score, name='Z_Score'))
fig.update_layout(title_text='Residuals Z-Score', template='simple_white', width=1100)
fig.show()

In [23]:
# Plot prices with z-scores

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=assets.index, y=assets[asset1], name=asset1), row=1, col=1)
fig.add_trace(go.Scatter(x=assets.index, y=assets[asset2], name=asset2), row=1, col=1)
fig.add_trace(go.Scatter(x=z_score.index, y=z_score, name='Z_Score'), row=2, col=1)
fig.update_layout(title_text='Análise Z_Score', template='simple_white', height=600, width=900)
fig.show()

## Backtest

- Residuals = X2 - beta * X1

- Define z-score threshold

- z-score > threshold = Sell X2 and buy X1 (SHORT THE SPREAD)

- z-score < threshold = Buy X2 and sell X1 (LONG THE SPREAD)


In [24]:
# Download open prices for assets
assets_open = pd.DataFrame()

for i in tickers:

  df2 = yf.download(i, start=start, end=end)['Open']
  df2.rename(i, inplace=True)
  assets_open = pd.concat([assets_open, df2], axis=1)
  assets_open.index.name='Date'

assets_open

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,KO,PEP
Date,,
2020-01-02,55.320000,136.869995
2020-01-03,54.320000,135.460007
2020-01-06,54.650002,135.300003
2020-01-07,54.450001,136.000000
2020-01-08,54.270000,134.460007
...,...,...
2022-12-23,63.500000,180.910004
2022-12-27,63.930000,183.279999
2022-12-28,64.459999,184.100006


In [25]:
#Define Backtest Parameters

cash = 100000
fees = 0.001

percentage_order_1 = 1
percentage_order_2 = 1

upper_open_threshold = 2
lower_open_threshold = -2

upper_close_threshold = 1
lower_close_threshold = -1

In [26]:
# Define the trading strategy

def pairs_trading_strategy(z_scores):
    position = np.zeros_like(z_scores)

    for i in range(len(z_scores)):
        if z_scores[i] > 2:
            position[i] = -1  # Enter short position
        elif z_scores[i] < -2:
            position[i] = 1  # Enter long position
        elif abs(z_scores[i]) < 1:
            position[i] = 0  # Zero position
        else:
            position[i] = position[i-1]

    return position

# Run the strategy
position = pairs_trading_strategy(z_score.values)

# Create DataFrame for results
result = pd.DataFrame({
    'z_scores': z_score,
    'position': position,
}, index=z_score.index)

result[asset1] = -result['position']
result[asset2] = result['position']
result

,z_scores,position,KO,PEP
Date,,,,
2020-01-02,-2.052286,1.0,-1.0,1.0
2020-01-03,-1.974858,1.0,-1.0,1.0
2020-01-06,-1.901093,1.0,-1.0,1.0
2020-01-07,-2.034067,1.0,-1.0,1.0
2020-01-08,-1.979164,1.0,-1.0,1.0
...,...,...,...,...
2022-12-23,0.852607,0.0,-0.0,0.0
2022-12-27,0.820570,0.0,-0.0,0.0
2022-12-28,0.874435,0.0,-0.0,0.0


In [27]:
# Plot z-score along with assets positions
fig = go.Figure()
fig.add_trace(go.Scatter(x=result.index, y=result['z_scores'], name='Z-Score'))
fig.add_trace(go.Scatter(x=result.index, y=result[asset1], name=f'Position {asset1}'))
fig.add_trace(go.Scatter(x=result.index, y=result[asset2], name=f'Position {asset2}'))
fig.update_layout(title_text='Backtest Positions', template='simple_white', height=600, width=1100)
fig.show()

In [28]:
#Create trading signals to open and close positions

vbt_short_signal = ((result['position'] == -1) & (result['position'].shift(1) != -1))
vbt_long_signal = ((result['position'] == 1) & (result['position'].shift(1) != 1))

vbt_close_short = ((result['position'] == 0) & (result['position'].shift(1) == -1))
vbt_close_long = ((result['position'] == 0) & (result['position'].shift(1) == 1))

In [29]:
# Create trades dataframe according to the signals

tickers_column = pd.Index([asset1, asset2], name='tickers')
vbt_trades = pd.DataFrame(index=assets.index, columns=tickers_column)
vbt_trades[asset1] = np.nan
vbt_trades[asset2] = np.nan

# Open position trades
vbt_trades.loc[vbt_short_signal, asset1] = percentage_order_1
vbt_trades.loc[vbt_long_signal, asset1] = -percentage_order_1
vbt_trades.loc[vbt_short_signal, asset2] = -percentage_order_2
vbt_trades.loc[vbt_long_signal, asset2] = percentage_order_2

# Close position trades
vbt_trades.loc[vbt_close_short, asset1] = 0
vbt_trades.loc[vbt_close_long, asset1] = 0
vbt_trades.loc[vbt_close_short, asset2] = 0
vbt_trades.loc[vbt_close_long, asset2] = 0

# Shift trades by one candle (avoid look-ahead bias)
vbt_trades = vbt_trades.vbt.fshift(1)

# Print trades
print(vbt_trades[~vbt_trades.isnull().any(axis=1)])

tickers               KO  PEP
Date                         
2020-01-03 00:00:00 -1.0  1.0
2020-03-16 00:00:00  0.0  0.0
2021-12-06 00:00:00  1.0 -1.0
2022-01-05 00:00:00  0.0  0.0
2022-10-13 00:00:00  1.0 -1.0
2022-12-07 00:00:00  0.0  0.0


In [30]:
# Run backtest
def portfolio_pairs_trading(group_results):

    return vbt.Portfolio.from_orders(
        assets,
        size=vbt_trades,
        price=assets_open,
        size_type='targetpercent',
        val_price=assets.vbt.fshift(1),
        init_cash=cash,
        fees=fees,
        cash_sharing=True,
        group_by=group_results,
        call_seq='auto',
        freq='d'
    )

vbt_pf = portfolio_pairs_trading(group_results=False)

In [32]:
# Plot orders of both assets
vbt_pf[0].plot(subplots='orders',
        title=asset1
    ).show()

vbt_pf[1].plot(subplots='orders',
        title=asset2
    ).show()

In [33]:
# Run backtest and print overall results
vbt_pf = portfolio_pairs_trading(group_results=True)
vbt_pf.plot(subplots='all', silence_warnings=True).show()

In [35]:
# Show backtest stats
vbt_pf.stats()

Start                         2020-01-02 00:00:00
End                           2022-12-30 00:00:00
Period                          756 days 00:00:00
Start Value                              100000.0
End Value                           122288.293079
Total Return [%]                        22.288293
Benchmark Return [%]                    35.858106
Max Gross Exposure [%]                   1.882825
Total Fees Paid                       1356.968253
Max Drawdown [%]                         3.469059
Max Drawdown Duration            26 days 00:00:00
Total Trades                                    6
Total Closed Trades                             6
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 50.0
Best Trade [%]                          22.060821
Worst Trade [%]                        -15.517609
Avg Winning Trade [%]                   16.247026
Avg Losing Trade [%]                    -9.712069
